# Text Preprocessing

In [1]:
import pandas as pd
import numpy as np

In [2]:
data1 = pd.read_csv('Resources/spam.csv', encoding = 'latin-1')
list = ['Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4']
data1 = data1.drop(list, axis=1)

data2 = pd.ExcelFile('Resources/indiandataset.xls')
data2 = data2.parse(0)
data2.head()

data = pd.concat([data1, data2])

In [3]:
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(data['msg'], data['label'], test_size = 0.30)

Y_train = pd.get_dummies(Y_train).values

In [4]:
from keras.preprocessing.text import text_to_word_sequence
temp = []
for i in range(len(X_train)):
    words = []
    rev = str(X_train.iloc[i])
    rev = text_to_word_sequence(rev)
    temp.append(rev)
X_train = temp

/home/manohar/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


# converting words to vectors

In [6]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

max_features = 3000

tokenizer = Tokenizer(num_words=max_features)
tokenizer.fit_on_texts(X_train)

X_train = tokenizer.texts_to_sequences(X_train)
X_train = pad_sequences(X_train)

In [14]:
tokenizer.texts_to_sequences('how are you')

[[], [1704], [604], [], [5], [113], [274], [], [363], [1704], [16]]

# designing nn classifier

In [7]:
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM

embed_dim = 128
lstm_out = 196


model = Sequential()
model.add(
    Embedding(max_features, embed_dim, input_length=X_train.shape[1]))
model.add(LSTM(lstm_out, dropout_U=0.2, dropout_W=0.2))
model.add(Dense(2, activation='softmax'))
model.compile(
    loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())

/home/manohar/.local/lib/python3.6/site-packages/ipykernel_launcher.py:11: UserWarning: Update your `LSTM` call to the Keras 2 API: `LSTM(196, dropout=0.2, recurrent_dropout=0.2)`
  # This is added back by InteractiveShellApp.init_path()


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 174, 128)          384000    
_________________________________________________________________
lstm_1 (LSTM)                (None, 196)               254800    
_________________________________________________________________
dense_1 (Dense)              (None, 2)                 394       
Total params: 639,194
Trainable params: 639,194
Non-trainable params: 0
_________________________________________________________________
None


# training

In [8]:
model.fit(X_train, Y_train, epochs = 10,batch_size = 20)

Epoch 1/10
7117/7117 [==============================] - 68s 10ms/step - loss: 0.1771 - acc: 0.9347
Epoch 2/10
7117/7117 [==============================] - 73s 10ms/step - loss: 0.0764 - acc: 0.9756
Epoch 3/10
7117/7117 [==============================] - 67s 9ms/step - loss: 0.0457 - acc: 0.9844
Epoch 4/10
7117/7117 [==============================] - 137s 19ms/step - loss: 0.0327 - acc: 0.9899
Epoch 5/10
7117/7117 [==============================] - 176s 25ms/step - loss: 0.0289 - acc: 0.9907
Epoch 6/10
7117/7117 [==============================] - 145s 20ms/step - loss: 0.0208 - acc: 0.9937
Epoch 7/10
7117/7117 [==============================] - 87s 12ms/step - loss: 0.0200 - acc: 0.9945
Epoch 8/10
7117/7117 [==============================] - 84s 12ms/step - loss: 0.0181 - acc: 0.9952
Epoch 9/10
7117/7117 [==============================] - 100s 14ms/step - loss: 0.0172 - acc: 0.9944
Epoch 10/10
7117/7117 [==============================] - 116s 16ms/step - loss: 0.0149 - acc: 0.9954


In [9]:
temp = []
for i in range(len(X_test)):
    words = []
    rev = str(X_test.iloc[i])
    rev = text_to_word_sequence(rev)
    temp.append(rev)
    
temp = tokenizer.texts_to_sequences(temp)
le = X_train.shape[1]
temp2 = [[0 for i in range(le)] for j in range(len(temp))]

for i in range(len(temp)):
    for j in range(len(temp[i])-1,-1,-1):
        temp2[i][le-1+j-len(temp[i])+1] = temp[i][j];

temp2 = np.array(temp2).reshape(len(temp),le,)

In [25]:
temp2.shape

(3051, 189)

In [10]:
Y_pr = model.predict(temp2)

Y_pred = []

for i in range(len(Y_pr)):
    if Y_pr[i][0] >= Y_pr[i][1]:
        Y_pred.append('ham')
    else:
        Y_pred.append('spam')
        
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(Y_test, Y_pred)  
print(cm)

[[2423   27]
 [  82  519]]


# wrongly predicted

In [28]:
import csv
file = open('output/wrongly_predicted2.csv', 'w')
wr_file = csv.writer(file)
wr_file.writerow(["label","msg"])

11

In [29]:
for i in range(len(Y_test)):    
    if not Y_test.iloc[i] == Y_pred[i]:
        wr_file.writerow([Y_test.iloc[i],X_test.iloc[i]])
file.close()

In [42]:
i= 787
print(type(Y_test.iloc[i]))
print(type(Y_pred[i]))

<class 'str'>
<class 'str'>
